In [1]:
import pandas as pd

from country_abbrev import *
from country_language import *
from pytrends.request import TrendReq

import os
import pycountry
import itertools

from googletrans import LANGCODES
import swifter

import trends_helpers
import numpy as np

In [2]:
currencies = pd.read_excel("countries-codes-and-currencies-(2020).xlsx")

In [3]:
keywords_list= list(set(currencies["Currency Name"]))
len(keywords_list)

141

In [4]:
countries_iso3 = currencies["Country Code"]
countries = [pycountry.countries.get(alpha_3=code).alpha_2 for code in countries_iso3]

In [5]:
# list of all unique languages:
unique_languages = pd.Series(list(set(list(itertools.chain(*country_language_dict.values())))), name='language')

# list of language codes from googletrans
langcodes = pd.DataFrame.from_dict(LANGCODES, orient='index', columns=['code'])
langcodes.index = langcodes.index.str.capitalize()

refugee_lang = unique_languages.to_frame().merge(langcodes, left_on='language', right_index=True, how='left')

refugee_lang.dropna(inplace=True)

refugee_lang_not_en = refugee_lang[refugee_lang['code'] != 'en']

In [6]:
translated_keyword_currency = refugee_lang_not_en['code'].swifter.apply(lambda x: trends_helpers.translate_keywords_list(lst = keywords_list, lang= x))

Pandas Apply:   0%|          | 0/83 [00:00<?, ?it/s]

In [7]:
columns = list(refugee_lang_not_en['code'])

df = pd.concat([pd.DataFrame(sublist, columns=[col]) for sublist, col in zip(translated_keyword_currency, columns)], axis=1)

df['en']=keywords_list

df.head()

,sm,ar,ceb,ha,be,mt,st,my,az,el,...,zu,th,fa,fi,jw,fr,kk,sr,ky,en
0,pauna a lepanona,ليرة لبنانية,lebanese pound,fam lebanon,ліванскі фунт,lira lebaniża,lebanese ponto,လက်ဘနွန်ပေါင်,livan funtu,λίρα λιβάνου,...,i-lebanese pound,ปอนด์เลบานอน,پوند لبنان,libanonin punta,pound libanon,livre libanaise,ливан фунты,либанска фунта,ливан фунту,Lebanese Pound
1,bhat,بهات,bhat,bhat,бхат,bhat,bhat,bhat,bhat,bhat,...,bhat,บาท,بات,bhat,bhat,bhat,бхат,бхат,bhat,Bhat
2,kuatemala quetzal,كوتزال غواتيمالا,guatemalan quetzal,guatemalan quetzal,гватэмальскі кетсаль,ketzal tal-gwatemala,quetzal ea guatemala,ဂွာတီမာလာ quetzal,qvatemalalı quetzal,quetzal της γουατεμάλας,...,i-quetzal yaseguatemala,เควตซัลกัวเตมาลา,کوتزال گواتمالا,guatemalan quetzal,quetzal guatemalan,quetzal guatémaltèque,гватемалалық кецаль,гватемалски квецал,guatemala quetzal,Guatemalan Quetzal
3,manat fou,مانات الجديدة,bag-ong manat,sabuwar manat,новы манат,manat ġdid,manat e ncha,manat အသစ်,yeni manat,νέο μανάτ,...,i-manat entsha,มนัส ใหม่,منات جدید,uusi manat,manat anyar,nouveau manat,жаңа манат,нови манат,жаңы манат,New Manat
4,djibouti franc,فرنك جيبوتي,djibouti franc,djibouti franc,джыбуційскі франк,djibouti franc,djibouti franc,ဂျီဘူတီ ဖရန့်,cibuti frankı,φράγκο τζιμπουτί,...,i-djibouti franc,ฟรังก์จิบูตี,فرانک جیبوتی,djiboutin frangi,frank djibouti,franc djiboutien,джибути франк,џибути франк,джибути франкы,Djibouti Franc


In [8]:
max_length = max(map(len, country_language_dict.values()))

data_padded = {key: arr + [np.nan] * (max_length - len(arr)) for key, arr in country_language_dict.items()}

langs = pd.DataFrame(data_padded)
langs = langs.T
langs = langs.reset_index()
langs = langs.rename(columns={'index': 'Country', 0:'lang1', 1:'lang2', 2:'lang3'})

# Apply the function to the 'Country' column
langs['ISO2'] = langs['Country'].apply(trends_helpers.get_iso2_country_code)

langs = langs.drop(columns=["Country"])
langs_long = pd.melt(langs, id_vars=['ISO2'], var_name='numlang', value_name='lang')
langs_long = langs_long.dropna()

In [9]:
langs_long = pd.merge(langs_long, refugee_lang_not_en, left_on="lang", right_on="language")

In [13]:
from pytrends.request import TrendReq
import time


# Set up the Google Trends API object
# pytrends = TrendReq()



# Loop through regions and keywords

for i, country in enumerate(langs_long["ISO2"]):
    languagecode= langs_long["code"][i]

    
    # Initialize an empty DataFrame to store the results
    results = pd.DataFrame()   

    for keyword in df[languagecode]:    
        print("Searching: " + str(keyword) + " in " + str(country))
        time.sleep(4)
        
        pytrends = TrendReq(tz=360, timeout=(10, 25), retries=2, backoff_factor=0.5,
            requests_args={'headers': {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64)'}})
        
        # Build the payload with the selected region, keyword and date range
        pytrends.build_payload(kw_list=[keyword], timeframe='all', geo=str(country))
        # Get the interest over time data for the selected keyword
        interest_over_time_df = pytrends.interest_over_time()

        # Check if the DataFrame is not empty
        if not interest_over_time_df.empty:
            interest_over_time_df= interest_over_time_df.reset_index()
            interest_over_time_df['trends_index'] = interest_over_time_df.iloc[:, 1] 
            interest_over_time_df = interest_over_time_df.drop(interest_over_time_df.columns[1], axis=1)

            interest_over_time_df = interest_over_time_df.drop(columns=["isPartial"], axis=1)
            interest_over_time_df["keyword"]=keyword
            interest_over_time_df["region"] = country
            
            # Append the results to the main dataframe
            results = pd.concat([results, interest_over_time_df])
    
    translation = {'keyword': df[langs_long["code"][0]], 'keyword_en': df["en"]}
    translation = pd.DataFrame(translation)
    results = pd.merge(results, translation, on="keyword", how="left")
    file_path = 'currency_partial results/original_lang/' + str(country) + '.csv'
    results.to_csv(file_path, index=False)
    
    time.sleep(120) # Wait two minutes until starting with next country


Searching: د لبنان پونډ in AF
Searching: بټ in AF
Searching: ګواتیمالا کویتزال in AF
Searching: نوی منات in AF
Searching: جیبوتي فرانک in AF
Searching: کولمبیا پیسو in AF
Searching: یورو in AF
Searching: لیسوتو لوټی in AF
Searching: کاناډايي ډالر in AF
Searching: سومالیا شلینګ in AF
Searching: لایبریا ډالر in AF
Searching: د تونس دینار in AF
Searching: د فیلیپین پیسو in AF
Searching: چیلي پیسو in AF
Searching: کوسټاریکا کولون in AF
Searching: روفيه in AF
Searching: افغانستان افغانی in AF
Searching: بیلیز ډالر in AF
Searching: بحرین دینار in AF
Searching: مکسیکو پیسو in AF
Searching: نوی اوګیا in AF
Searching: نایره in AF
Searching: کوچه in AF
Searching: مالدووا لیو in AF
Searching: هنګري forint in AF
Searching: ایتوپیا بیر in AF
Searching: pa'anga in AF
Searching: ملاګاسي آریري in AF
Searching: سینګ in AF
Searching: جاپاني ین in AF
Searching: د یمن ریال in AF
Searching: د ګمبیا دلاسي in AF
Searching: هندي روپۍ in AF
Searching: د متحده اماراتو درهم in AF
Searching: سویډني کرونا in AF
Se

RetryError: HTTPSConnectionPool(host='trends.google.com', port=443): Max retries exceeded with url: /trends/api/explore?hl=en-US&tz=360&req=%7B%22comparisonItem%22%3A+%5B%7B%22keyword%22%3A+%22dollar+barbados%22%2C+%22time%22%3A+%22all%22%2C+%22geo%22%3A+%22AL%22%7D%5D%2C+%22category%22%3A+0%2C+%22property%22%3A+%22%22%7D (Caused by ResponseError('too many 429 error responses'))